In [ ]:
import socket
import threading
from PyQt5.QtWidgets import (
    QApplication, QVBoxLayout, QTextEdit, QLineEdit, QPushButton, QWidget, QMessageBox
)
from PyQt5.QtCore import pyqtSignal
from cryptography.fernet import Fernet


# Server Configuration
HOST = '127.0.0.1'
PORT = 12345


class ChatClient(QWidget):
    # Signal to update the chat display safely from a different thread
    message_received_signal = pyqtSignal(str)

    def __init__(self):
        super().__init__()
        self.client_socket = None
        self.cipher = None
        self.username = None
        self.init_ui()

    def init_ui(self):
        """Initialize the GUI layout."""
        self.setWindowTitle("Chat Client")

        # Layout
        layout = QVBoxLayout()

        # Chat Display
        self.chat_display = QTextEdit()
        self.chat_display.setReadOnly(True)
        layout.addWidget(self.chat_display)

        # Message Input
        self.message_input = QLineEdit()
        self.message_input.setPlaceholderText("Type your message here...")
        layout.addWidget(self.message_input)

        # Send Button
        send_button = QPushButton("Send")
        send_button.clicked.connect(self.send_message)
        layout.addWidget(send_button)

        self.setLayout(layout)

        # Connect message received signal to display update
        self.message_received_signal.connect(self.update_chat_display)

    def connect_to_server(self):
        """Connect to the chat server."""
        try:
            self.client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            self.client_socket.connect((HOST, PORT))

            # Receive the encryption key from the server
            key = self.client_socket.recv(1024)
            self.cipher = Fernet(key)

            # Prompt for username and password
            self.username = self.get_credentials()
            if self.username:
                # Start a thread to listen for incoming messages
                threading.Thread(target=self.receive_messages, daemon=True).start()
            else:
                QMessageBox.warning(self, "Authentication Failed", "Unable to log in or register.")
                self.close()

        except Exception as e:
            QMessageBox.critical(self, "Connection Error", f"Failed to connect to server: {e}")
            self.close()

    def get_credentials(self):
        """Prompt the user for credentials and authenticate with the server."""
        username, password = self.prompt_credentials()
        if username and password:
            creds = f"{username}:{password}"
            self.client_socket.send(creds.encode())

            # Wait for server response
            response = self.client_socket.recv(1024).decode()
            if response == "INVALID":
                QMessageBox.warning(self, "Invalid Credentials", "Username or password is incorrect.")
                return None
            else:
                return username
        return None

    def prompt_credentials(self):
        """Prompt the user for their username and password."""
        username, ok = QInputDialog.getText(self, "Username", "Enter your username:")
        if not ok or not username.strip():
            return None, None

        password, ok = QInputDialog.getText(self, "Password", "Enter your password:", QLineEdit.Password)
        if not ok or not password.strip():
            return None, None

        return username.strip(), password.strip()

    def send_message(self):
        """Send an encrypted message to the server."""
        message = self.message_input.text().strip()
        if message:
            try:
                if ":" in message:
                    recipient, msg = message.split(":", 1)
                    final_message = f"{recipient}:{msg}"
                else:
                    final_message = message

                encrypted_message = self.cipher.encrypt(final_message.encode())
                self.client_socket.send(encrypted_message)
                self.message_input.clear()
            except Exception as e:
                QMessageBox.critical(self, "Send Error", f"Failed to send message: {e}")
        else:
            QMessageBox.warning(self, "Warning", "Message cannot be empty!")

    def receive_messages(self):
        """Continuously listen for and decrypt messages from the server."""
        while True:
            try:
                encrypted_message = self.client_socket.recv(1024)
                if not encrypted_message:
                    break
                message = self.cipher.decrypt(encrypted_message).decode()
                self.message_received_signal.emit(message)  # Emit signal to update the chat display
            except Exception as e:
                QMessageBox.critical(self, "Receive Error", f"Error receiving message: {e}")
                break

    def update_chat_display(self, message):
        """Update the chat display with a new message."""
        self.chat_display.append(message)

    def closeEvent(self, event):
        """Handle window close events to clean up resources."""
        try:
            if self.client_socket:
                self.client_socket.close()
        except:
            pass
        event.accept()


if __name__ == "__main__":
    import sys
    from PyQt5.QtWidgets import QInputDialog, QLineEdit

    app = QApplication([])

    client = ChatClient()
    client.connect_to_server()
    client.show()

    sys.exit(app.exec_())
